In [ ]:
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
file_path = "crypto_data.csv"
df_crypto = pd.read_csv(file_path,index_col = 0)
df_crypto.head()

In [ ]:
df_crypto = df_crypto[df_crypto.IsTrading.eq(True)]
df_crypto.shape

In [ ]:
pd.isna(df_crypto['Algorithm'])

In [ ]:
df_crypto = df_crypto.drop(["IsTrading"],axis = 1)

In [ ]:
df_crypto = df_crypto.dropna(how='any',axis=0) 
df_crypto

In [ ]:
df_crypto = df_crypto[df_crypto.TotalCoinsMined > 0]
df_crypto

In [ ]:
names = df_crypto.filter(['CoinName'], axis=1)
names

In [ ]:
df_crypto = df_crypto.drop(['CoinName'],axis = 1)
df_crypto

In [ ]:
crypto = pd.get_dummies(df_crypto['Algorithm'])
dummy = pd.get_dummies(df_crypto['ProofType'])
combined = pd.concat([crypto,dummy],axis =1)
df = df_crypto.merge(combined,left_index = True,right_index = True)
df = df.drop(['Algorithm','ProofType'],axis = 1)
df

In [ ]:
df.head()

In [ ]:
df_scaled = StandardScaler().fit_transform(df)
print(df_scaled)

In [ ]:
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)
df_pca

In [ ]:
pcs_df = pd.DataFrame(
    data = df_pca, columns = ['PC1','PC2','PC3'], index = df_crypto.index
)
pcs_df

In [ ]:
inertia = []
k = list(range(1, 11))

   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

elbow_data = {"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")


In [ ]:
model = KMeans(n_clusters=4, random_state=0)

model.fit(pcs_df)

predictions = model.predict(pcs_df)

In [ ]:
clustered_df = pd.concat([df_crypto,pcs_df],axis =1)

clustered_df['CoinName'] = names['CoinName']

clustered_df["Class"]= model.labels_

print(clustered_df.shape)
clustered_df.head(10)

In [ ]:
fig = px.scatter_3d(
    clustered_df,
    hover_name="CoinName",
    hover_data=["Algorithm"],
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
clustered_df.hvplot.table(sortable=True, selectable=True)

In [ ]:
print(clustered_df.count)

In [ ]:
scaled =MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])
print(scaled)

In [ ]:
new_df = pd.DataFrame(
    data = scaled,columns = ["TotalCoinSupply","TotalCoinsMined"], index = clustered_df.index
)

new_df = pd.concat([new_df,clustered_df['CoinName']],axis =1)


plot_df = pd.concat([new_df,clustered_df["Class"]],axis =1)

plot_df.head(10)

In [ ]:
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class",hover_cols = ["CoinName"])